In [ ]:
%matplotlib inline


Train a Classifier using PyTorch
=====================

In this exercise you will implement and train the L-Layer NN of the previous exercise but this time using PyTorch.



Training an image classifier
----------------------------

You will do the following steps in order:

1. Load the cars dataset
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

### 1. Loading the cars dataset


In the previous exercise, you used a proprietary loader to load a dataset from a HDF5 file. This time, you will use the datset loading functionality provided by torchvision.



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler, DataLoader, random_split
import numpy as np

np.random.seed(0)
torch.manual_seed(0)

The dataset files are located in the folder dataset_cars. The sub-folder "cars" contains all car images the sub-folder "non_car" contains all images not showing cars.

Using torchvision, a dataset can be loaded from a folder using the ````ImageFolder```` class (https://pytorch.org/vision/0.11/datasets.html?highlight=imagefolder#torchvision.datasets.ImageFolder).

Furthermore, you can define a transform which is applied to each image. By deafult, the dataset returns PIL images. However, the input for NNs in PyTorch are of type Tensor. Therefore, you have to apply the ````ToTensor()```` transform (see https://pytorch.org/vision/stable/transforms.html for details).

For testing the training result, you need to split the data into a training and a test set. The training set should contain most of the data (80%), the test set the rest. PyTorch offers the ````random_split()```` function to do this. In order to get repeatable results, you have to set the ````generator```` parameter to ````torch.Generator().manual_seed(0)````. Documentation can be found here: https://pytorch.org/docs/stable/data.html



In [ ]:
# YOUR CODE STARTS HERE
# define a transformation for the dataset, use ToTensor()
transform = 

# define the dataset, use the ImageFolder class, apply the transform
dataset = 

# Split the dataset into a train and test set. The train set should contain 80% of the images, the test set the remainder
train_set_size = 
test_set_size = 
train_set, test_set = 
# YOUR CODE ENDS HERE

print(f"Number of images in training set: {len(train_set)}")
print(f"Number of images in training set: {len(test_set)}")
classes = tuple(dataset.class_to_idx)
print(f"Classes in dataset: {classes}")

**Expected Output:**

````Number of images in training set: 296
Number of images in training set: 75
Classes in dataset: ('car', 'non_car')````

Next, you need to specify a loader for the dataset. A loader specify how the data is provided as input for the NN. For example, you can define to load a certain number of images at once (called mini-batch). As we do batch learning, you have to load the whole training set at once. For the test set, we want to load only one image per iteration.

Define a ````train_loader```` and a ````test_loader```` using the ````DataLoader```` class. You have to specify the ````dataset```` and ````batch_size```` parameters. Details can be found here: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

In [ ]:
# YOUR CODE STARTS HERE
# The train_loader should work on train_set and use the size of train_set as batch size
train_loader = 
# The test_loader should work on test_set and use 1 as batch size
test_loader = 
#YOUR CODE ENDS HERE

print(f"train_loader works on a dataset of {len(train_loader.dataset)} images and has a batch_size of {train_loader.batch_size}.")
print(f"test_loader works on a dataset of {len(test_loader.dataset)} images and has a batch_size of {test_loader.batch_size}.")

**Expected Output:**

````train_loader works on a dataset of 296 images and has a batch_size of 296.
test_loader works on a dataset of 75 images and has a batch_size of 1.````

Let us show some of the training images, for fun.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# functions to show an image


def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
sample_idx = random.sample(range(len(images)),4)
plot_images = [images[i] for i in sample_idx]

# show images
imshow(torchvision.utils.make_grid(plot_images))
# print labels
print(' '.join('%11s' % classes[labels[j]] for j in sample_idx))

### 2. Define a L-layer Deep Neural Network

Now we want to implement the L-layer network from the previous exercise in PyTorch. The network is depicted in this image:

<img src="images/LlayerNN.png" style="width:800px;height:324px;">
<caption><center> <u>Figure 3</u>: L-layer neural network. <br> The model can be summarized as: [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID</center></caption>

As in the last exercise, we choose ````L=4````. The input is a tensor of size (m, 12288). The layer dimensions should bes as followed

| Layer | Number of units |
|---|---|
| Hidden 1 | 30 |
| Hidden 2 | 10 |
| Hidden 3 | 7 |
| Output | 1 |

In PyTorch, most layers are already implemented. You need fully connected (linear) layers (````torch.nn.Linear()````), ReLU layers (````torch.nn.functional.relu()````) and a Sigmoid layer (````torch.nn.Sigmoid()````). For defining a NN, you have to create a class which is deroved from ````nn.Module````. In the constructor, the layers are defined. You also need to define the ````forward()```` function which represents the forward propagation and specifies how the layers are connected. The first three layers use a ReLU function, it can be used like this: ````F.relu(self.lx(x))````. You can find further information here: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # YOUR CODE STARTS HERE
        # Define the 4 layers, use Linear() and Sigmoid()
        self.l1 = 
        self.l2 = 
        self.l3 = 
        self.l4 = 
        self.activation = 
        # YOUR CODE ENDS HERE

    def forward(self, x):
        # YOUR CODE STARTS HERE
        # Define the forward propagation, x is both input and output of each layer. Add the ReLU function to each linear layer
        x = 
        x = 
        x = 
        x = 
        # YOUR CODE ENDS HERE
        return self.activation(x)


net = Net()

print(net)

**Expected Output:**

````Net(
  (l1): Linear(in_features=12288, out_features=30, bias=True)
  (l2): Linear(in_features=30, out_features=10, bias=True)
  (l3): Linear(in_features=10, out_features=7, bias=True)
  (l4): Linear(in_features=7, out_features=1, bias=True)
  (activation): Sigmoid()
)````

### 3. Define a Loss function and optimizer

Next, you have to choose the loss function and the optimization algorithm. We want to use the logistic regression loss function. In PyTorch, this is implement as ````nn.BCELoss()````. As optimizer, we use gradient descent (````torch.optim.SGD()````). It takes the parameters of your network as input (````net.parameters()````). Furthermore, you have to specify the learning rate (````lr=0.0075````). The ````momentum```` parameter should be set to 0.



In [ ]:
import torch.optim as optim

# YOUR CODE STARTS HERE
criterion = 
optimizer = 
# YOUR CODE ENDS HERE

print(criterion)
print(optimizer)

**Expected Output:**

````BCELoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0075
    momentum: 0
    nesterov: False
    weight_decay: 0
)````

### 4. Train the network


Now you are going to train your model! First, we load the data using ````train_loader````. The image tensors are of shape (296, 3, 64, 64) but the net expects a an input of (296, 12288). This means, we have to flatten the images. For cost computation, the labels need to be of type float, the apporpriate conversion is also done.

For training, you have to implement a loop with 2500 iterations over the data. In each step, 
 1. Outputs are computed by calling ````net()````
 2. The loss is calculated by calling the criterion with the output of the net and the lables as input
 3. The gradients are computed via backward propagation by calling ````backward()```` on the loss
 4. One gradient descent step is done by calling the ````step()```` function of the optimizer



In [ ]:
# these lines are only needed when training on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#net.to(device)

for data in train_loader:
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    #inputs, labels = data[0].to(device), data[1].to(device)
    # reshape the inputs and convert the labels from int to float
    inputs_flatten = inputs.reshape(inputs.shape[0],-1)
    labels_flatten = labels.reshape(labels.shape[0],1).float()
    
    # these lines are only needed when training on GPU
    #inputs_flatten = inputs_flatten.to(device)
    #labels_flatten = labels_flatten.to(device)

    for i in range(2500):  # loop over the dataset multiple times
        # zero the parameter gradients
        optimizer.zero_grad()

        #YOUR CODE STARTS HERE
        # forward + backward + optimize
        outputs = 
        loss = 
        #YOUR CODE ENDS HERE
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 100 == 0:    # print every 100 iterations
            print(f'loss after {i} iterations: {loss.item():.5f}')

print('Finished Training')

**Expected Output:**

````loss after 0 iterations: 0.09601
loss after 100 iterations: 0.09404
loss after 200 iterations: 0.09173
loss after 300 iterations: 0.08909
loss after 400 iterations: 0.08605
loss after 500 iterations: 0.08269
loss after 600 iterations: 0.07890
loss after 700 iterations: 0.07486
loss after 800 iterations: 0.07112
loss after 900 iterations: 0.06744
loss after 1000 iterations: 0.06413
loss after 1100 iterations: 0.06083
loss after 1200 iterations: 0.05782
loss after 1300 iterations: 0.05501
loss after 1400 iterations: 0.05236
loss after 1500 iterations: 0.04982
loss after 1600 iterations: 0.04745
loss after 1700 iterations: 0.04522
loss after 1800 iterations: 0.04313
loss after 1900 iterations: 0.04107
loss after 2000 iterations: 0.03899
loss after 2100 iterations: 0.03700
loss after 2200 iterations: 0.03523
loss after 2300 iterations: 0.03357
loss after 2400 iterations: 0.03209
Finished Training````

### 5. Test the network on the test data

You have trained the network for 2500 passes over the training dataset.
Now you need to check if the network has learnt anything at all.

You will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, you add the sample to the list of correct predictions.

Okay, first step. Let us display some images from the test set to get familiar.



In [ ]:
dataiter = iter(test_loader)
images = []
labels = []
for i in range(4):
    image, label = dataiter.next()
    images.append(image[0])
    labels.append(label[0])

# print images
imshow(torchvision.utils.make_grid(images))
print(' '.join('%11s' % classes[j] for j in labels))

Okay, now let us see what the neural network thinks the first example above is:



In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
images_flatten = images.reshape(images.shape[0],-1)
outputs = net(images_flatten)
print(outputs)
predicted = round(outputs.data.item())

print(f'Predicted: {classes[predicted]}')

The output is a probability for being 'non_car', which means a car image should have a value close to 0 to be classified correctly. As you can see, this is the case for the example above.



Let us look at how the network performs on the whole test set.


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images_flatten = images.reshape(images.shape[0],-1)
        outputs = net(images_flatten)
        predicted = round(outputs.data.item())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

That looks way better than chance, which is 50% accuracy (randomly picking
a class out of 2 classes).
If you compare the accuracy to the previous exercise, you can see that the values is similar. The difference mainly comes from the different train/test split in the dataset.



To speed things up, we can also train the network on a GPU. This only works if you have a GPU in you PC that supports CUDA. It also won't work on the Jupyter Hub or on the virtual machine.

Training on GPU
----------------
Just like how you transfer a Tensor onto the GPU, you transfer the neural
net onto the GPU.

Let's first define our device as the first visible cuda device if we have
CUDA available:



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

Only continue if you see something like ````cuda:0```` which means you have an appropriate GPU. In this case, you can uncomment the lines makred "Only needed when training on GPU" in the training cell above and re-run the training. Depending on your hardware, you might see an increase in training speed.'
